# Encontrando parámetros $\alpha$ y $\beta$ que se ajusten a la función

$$
(c/a)(V) = (c/a)(V_0) + \alpha(1-V/V_0) + \beta(1-V/V_0)^2,
$$

de los datos en la tabla `data/processed/vol_energia_presion.csv`.

## Importando liberías a utilizar

In [274]:
import os
import pandas as pd
import numpy as np

## Definiendo directorio principal

In [275]:
directorio_principal = '/home/marcocarmonaga/Documents/MoB_2-Study-Properties_Quantum_expresso/notebooks-scripts'
os.chdir(directorio_principal)

## Cambiando a carpeta sobre la que se trabajará (directorio secundario)

In [276]:
energia_total_directory = '../data/processed/'
os.chdir(energia_total_directory)

## Listando archivos

In [277]:
files = os.listdir()
files

['volumen-presion.csv',
 'valores_minimos.csv',
 'energia-total-por-volumen',
 'eos',
 'valores-minimos-uniendo-177_09.csv']

## Definiendo variables a utilizar

In [278]:
# V_0
V0 = 177.91

# (c/a)(V0)
c_a_V0 = 1.10154

## Encontrando parámetros $\alpha$ y $\beta$

La idea está en encontrar los parámetros $\alpha$, $\beta$; tales que se cumpla lo siguiente:

$$
\begin{pmatrix}
1-V_{i}/V_0 & (1-V_{i}/V_0)^2\\
\vdots & \vdots
\end{pmatrix}
\begin{pmatrix}
\alpha \\
\beta
\end{pmatrix}=
\begin{pmatrix}
(c/a)(V_{i}) - (c/a)(V_0)\\
\vdots
\end{pmatrix}
$$

### Obteniendo cada uno de los términos

In [279]:
df = pd.read_csv('valores-minimos-uniendo-177_09.csv')
df.drop(columns=['Energia_min'], inplace=True)
df['termino_lineal'] = df['Volumen'].apply(lambda x: 1 - (x / V0))
df['termino_cuadratico'] = df['Volumen'].apply(lambda x: (1 - (x / V0))**2)
df['diferencia'] = df['c_a_min'].apply(lambda x: x-c_a_V0)
df

,Volumen,c_a_min,termino_lineal,termino_cuadratico,diferencia
0,159.30,1.08425,0.104603,0.010942,-0.01729
1,162.84,1.08725,0.084706,0.007175,-0.01429
2,166.38,1.09041,0.064808,0.004200,-0.01113
3,169.92,1.09371,0.044910,0.002017,-0.00783
4,173.46,1.09710,0.025013,0.000626,-0.00444
5,177.00,1.10063,0.005115,0.000026,-0.00091
6,177.91,1.10154,0.000000,0.000000,0.00000
7,180.54,1.10426,-0.014783,0.000219,0.00272
8,184.08,1.10797,-0.034680,0.001203,0.00643
9,187.62,1.11184,-0.054578,0.002979,0.01030


## Eliminando columnas innecesarias para el ajuste.

In [280]:
df_copy = df.copy()
df_copy.drop(columns=['Volumen','c_a_min'],inplace=True)

## Separando dataframe y convirtiendo en matriz

In [281]:
A = df_copy.iloc[:,0:2].values
b = df_copy['diferencia'].values

print(A,b)

[[ 1.04603451e-01  1.09418820e-02]
 [ 8.47057501e-02  7.17506410e-03]
 [ 6.48080490e-02  4.20008322e-03]
 [ 4.49103479e-02  2.01693935e-03]
 [ 2.50126468e-02  6.25632502e-04]
 [ 5.11494576e-03  2.61626701e-05]
 [ 0.00000000e+00  0.00000000e+00]
 [-1.47827553e-02  2.18529855e-04]
 [-3.46804564e-02  1.20273406e-03]
 [-5.45781575e-02  2.97877528e-03]
 [-7.44758586e-02  5.54665351e-03]
 [-9.43735597e-02  8.90636876e-03]] [-0.01729 -0.01429 -0.01113 -0.00783 -0.00444 -0.00091  0.       0.00272
  0.00643  0.0103   0.01419  0.01821]


## Entontrar parámetros $\alpha$ y $\beta$ resolviendo lo siguiente:

$$
x = A^{-1}b
$$

donde $x=[\alpha\ \beta]$

Sin embargo, al tratarse de una matriz no cuadrada, procederemos a hacer una descomposición QR, para así proceder a resolver el siguiente sistema:

$$
x = R^{-1}Q^{T}b
$$

In [282]:
Q, R = np.linalg.qr(A)

alpha, beta = np.linalg.solve(R, np.dot(Q.T,b))
print('alpha =',alpha.round(5))
print('beta =',beta.round(5))

alpha = -0.18024
beta = 0.13883


## Conclusión

$$
\boxed{\therefore\ (c/a)(V) = (c/a)(V_0) + \alpha(1-V/V_0) + \beta(1-V/V_0)^2}
$$

con $\alpha = -0.18024$, $\beta = 0.13883$ y $(c/a)(V_0)=1.10154$

## Regresando a directorio principal

In [283]:
os.chdir(directorio_principal)